In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib as plt

ModuleNotFoundError: No module named 'matplotlib'

In [5]:
test_data = pd.read_csv("dataset_test.csv")
test_labels = test_data["Class"]
test_data = test_data.drop("Class", axis=1)
train_data = pd.read_csv("dataset_train.csv")
train_labels = train_data["Class"]
train_data = train_data.drop("Class", axis=1)

In [7]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, 0.2)

NameError: name 'train_test_split' is not defined